In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.stats as st

# ploting
import matplotlib.pyplot as plt

In [2]:
notes = set() 
input_folder = "kaggle/input/linking-writing-processes-to-writing-quality/"

In [3]:
train_logs = pd.read_csv(input_folder + "train_logs.csv",delimiter = ",",header = 0)
train_logs_indexed = train_logs.set_index(['id', 'event_id'])
train_scores = pd.read_csv(input_folder +"train_scores.csv", delimiter = ",", header = 0)
scores = pd.Series(data = train_scores['score'].values, index = train_scores['id'].values, name = 'score')

In [4]:
train_logs.head(2)

,id,event_id,down_time,up_time,action_time,activity,down_event,up_event,text_change,cursor_position,word_count
0,001519c8,1,4526,4557,31,Nonproduction,Leftclick,Leftclick,NoChange,0,0
1,001519c8,2,4558,4962,404,Nonproduction,Leftclick,Leftclick,NoChange,0,0


In [5]:
sum = 0
for i in (np.arange(0.5, 6.5, 0.5)):
    print("score = {}: length = {}".format(i,scores[scores == i].shape[0]))
    sum += scores[scores == i].shape[0]
print("sum: {}".format(sum))

score = 0.5: length = 5
score = 1.0: length = 35
score = 1.5: length = 69
score = 2.0: length = 92
score = 2.5: length = 201
score = 3.0: length = 336
score = 3.5: length = 486
score = 4.0: length = 501
score = 4.5: length = 402
score = 5.0: length = 179
score = 5.5: length = 128
score = 6.0: length = 37
sum: 2471


In [6]:
''' 
check mean of cursor position going back . Does high essay score tend to go back more from previous centences
than low essay scores? 
'''
def filter_negative(x):
    how = True
    if(x < 0):
        how = True
    else:
        how = False
    return how
    
def neg_movement_statistic(x):
    n = np.diff(np.log(x+1))
    n1 = np.array(list(filter(filter_negative, n)))
    return st.skew(n1)

def filter_positive(x):
    how = True
    if(x > 0):
        how = True
    else:
        how = False
    return how
    
def pos_movement_statistic(x):
    n = np.diff(np.log(x+1))
    n1 = np.array(list(filter(filter_positive, n)))
    return st.skew(n1)

In [7]:
# on average, how many times does the cursor go back:
stat_cursor_backward= train_logs.groupby('id')['cursor_position'].aggregate(neg_movement_statistic)
stat_cursor_backward.name = "stat_cursor_backward"
stat_cursor_forward= train_logs.groupby('id')['cursor_position'].aggregate(pos_movement_statistic)
stat_cursor_forward.name = "stat_cursor_forward"

In [8]:
z = pd.merge(stat_cursor_backward, stat_cursor_forward, left_index = True, right_index = True)
z = pd.merge(z, scores, left_index = True, right_index = True)
z.corr()

,stat_cursor_backward,stat_cursor_forward,score
stat_cursor_backward,1.000000,-0.541942,-0.313955
stat_cursor_forward,-0.541942,1.000000,0.336884
score,-0.313955,0.336884,1.000000


In [9]:
mean_cursor_forward

NameError: name 'mean_cursor_forward' is not defined